# Project Football

In this project we checked historical data about football matches.

_SOL_: I made a change about some business questions.

## Data Collection

In [69]:
import pandas as pd

# get data from kaggle
football_train_df = pd.read_csv('./data/football_train.csv')
football_target_df = pd.read_csv('./data/football_target.csv')

# print shapes
print("train shape:",football_train_df.shape)
print("target shape:",football_target_df.shape)


/tmp/ipykernel_8665/2508047730.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  football_train_df = pd.read_csv('./data/football_train.csv')


train shape: (110938, 190)
target shape: (110938, 3)


In [70]:
football_target_df.set_index('id', inplace=True)
football_train_df.set_index('id', inplace=True)

In [71]:
# join football_train_df and football_target_df (only score column)
football_target_df = football_target_df[['score']]
football_train_df = football_train_df.join(football_target_df)
print(football_train_df.shape)

(110938, 190)


## Data Preprocessing

### Data Cleaning

We remove, fill, replace, ...

In [72]:
# remove every team that has less than 10 matches
teams_less_10_matches = football_train_df['home_team_name'].value_counts() < 10
teams_less_10_matches = teams_less_10_matches[teams_less_10_matches].index

# remove teams from teams_less_10_matches
football_train_df = football_train_df[~football_train_df['home_team_name'].isin(teams_less_10_matches)]
football_train_df = football_train_df[~football_train_df['away_team_name'].isin(teams_less_10_matches)]
print(football_train_df.shape)

(79055, 190)


In [73]:
# Extract diccionary with the league_id and league_name and drop league_name from the dataframes
league_dict = football_train_df[['league_id', 'league_name']].drop_duplicates().set_index('league_id').to_dict()['league_name']
print(football_train_df.shape)


(79055, 190)


Se eliminaron los datos del coach debido a que habian muy poquitos que cumplian con esto y solamente añadia ruido \
Eliminamos todos los equipos que tuvieran menos de 10 partidos en el historico \
Eliminamos aquellas filas que tengan menos del 50% de información disponible


### Filtros opcionales (ejecutar sin eliminar los equipos)

In [ ]:
# remove every column has "coach" in his name
football_train_df = football_train_df[football_train_df.columns.drop(list(football_train_df.filter(regex='coach', axis=1)))]
print(football_train_df.shape)

(86969, 168)


In [ ]:
# remove every row has more than 50% of missing values
football_train_df = football_train_df.dropna(thresh=0.5*len(football_train_df), axis=1)
print(football_train_df.shape)


In [ ]:
# remove every row has more than 70% of missing values
football_train_df = football_train_df.dropna(thresh=0.7*len(football_train_df.columns), axis=0)
print(football_train_df.shape)

In [57]:
# code to remove duplicates
football_train_df = football_train_df.drop_duplicates()

In [82]:
# imprimir columnas
football_train_df.columns.to_list()

['target',
 'home_team_name',
 'away_team_name',
 'match_date',
 'league_name',
 'league_id',
 'is_cup',
 'home_team_coach_id',
 'away_team_coach_id',
 'home_team_history_match_date_1',
 'home_team_history_match_date_2',
 'home_team_history_match_date_3',
 'home_team_history_match_date_4',
 'home_team_history_match_date_5',
 'home_team_history_match_date_6',
 'home_team_history_match_date_7',
 'home_team_history_match_date_8',
 'home_team_history_match_date_9',
 'home_team_history_match_date_10',
 'home_team_history_is_play_home_1',
 'home_team_history_is_play_home_2',
 'home_team_history_is_play_home_3',
 'home_team_history_is_play_home_4',
 'home_team_history_is_play_home_5',
 'home_team_history_is_play_home_6',
 'home_team_history_is_play_home_7',
 'home_team_history_is_play_home_8',
 'home_team_history_is_play_home_9',
 'home_team_history_is_play_home_10',
 'home_team_history_is_cup_1',
 'home_team_history_is_cup_2',
 'home_team_history_is_cup_3',
 'home_team_history_is_cup_4',
 'h

### Data Preparation

In [81]:
from sklearn.preprocessing import LabelEncoder

# change datetime format
# filter date columns and change datatype
for col in football_train_df.filter(regex='date', axis=1).columns:
    football_train_df[col] = pd.to_datetime(football_train_df[col])

# Set league_name as category
football_train_df['league_name'] = football_train_df['league_name'].astype('category')

# Label encoding target
le = LabelEncoder()
football_train_df['target'] = le.fit_transform(football_train_df['target'])

# Label encoding is_coup
football_train_df['is_cup'] = le.fit_transform(football_train_df['is_cup'])

# Separate score column # - # in two columns home_score and away_score and merge it with football_train_df
football_train_df['home_score'] = football_train_df['score'].str.split('-', expand=True)[0].astype(int)
football_train_df['away_score'] = football_train_df['score'].str.split('-', expand=True)[1].astype(int)
football_train_df.drop('score', axis=1, inplace=True)



## Feature Engineering


In [119]:
# Feature Engineering
football_reduced_df = football_train_df[[
    'home_team_name', 
    'away_team_name', 
    'match_date', 
    'league_name',
    'is_cup', 
    'home_score', 
    'away_score',
    'target']].copy()

# Calculate last home team match
# find the days between the last match of the home team
football_reduced_df['home_days_betwent_last_match'] = (football_reduced_df['match_date'] - football_train_df['home_team_history_match_date_1']).dt.days
football_reduced_df['home_days_betwent_last_match'].head(10)

# home average goals in last 10 matches (columns home_team_history_goal_1 to home_team_history_goal_10)
football_reduced_df['home_avg_home_goal_last_10'] = football_train_df.filter(regex='home_team_history_goal', axis=1).mean(axis=1)
football_reduced_df['home_avg_opponent_goal_last_10'] = football_train_df.filter(regex='home_team_history_opponent_goal', axis=1).mean(axis=1)

football_reduced_df['away_avg_home_goal_last_10'] = football_train_df.filter(regex='away_team_history_goal', axis=1).mean(axis=1)
football_reduced_df['away_avg_opponent_goal_last_10'] = football_train_df.filter(regex='away_team_history_opponent_goal', axis=1).mean(axis=1)
# Aqui podemos hacer feature engineering para obtener la cantidad de partidos jugados por cada equipo

# Habra algun mes en donde el rendimiento se vea aectado?
# create columns by month based on date
football_reduced_df['month'] = football_reduced_df['match_date'].dt.month

# how many matches did win the home/away team in the last 10 matches
home_history_goals = football_train_df.filter(regex='home_team_history_goal', axis=1).to_numpy()
home_history_oponent_goals = football_train_df.filter(regex='home_team_history_opponent_goal', axis=1).to_numpy()
football_reduced_df['home_wins_last_10'] = (home_history_goals > home_history_oponent_goals).sum(axis=1)
football_reduced_df['home_dawn_last_10'] = (home_history_goals == home_history_oponent_goals).sum(axis=1)
football_reduced_df['home_lose_last_10'] = (home_history_goals < home_history_oponent_goals).sum(axis=1)

away_history_goals = football_train_df.filter(regex='away_team_history_goal', axis=1).to_numpy()
away_history_oponent_goals = football_train_df.filter(regex='away_team_history_opponent_goal', axis=1).to_numpy()
football_reduced_df['away_wins_last_10'] = (away_history_goals > away_history_oponent_goals).sum(axis=1)
football_reduced_df['away_dawn_last_10'] = (away_history_goals == away_history_oponent_goals).sum(axis=1)
football_reduced_df['away_lose_last_10'] = (away_history_goals < away_history_oponent_goals).sum(axis=1)

# mean team rating and oponent rating
football_reduced_df['home_avg_rating_last_10'] = football_train_df.filter(regex='home_team_history_rating', axis=1).mean(axis=1)
football_reduced_df['home_avg_opponent_rating_last_10'] = football_train_df.filter(regex='home_team_history_opponent_rating', axis=1).mean(axis=1)

football_reduced_df['away_avg_rating_last_10'] = football_train_df.filter(regex='away_team_history_rating', axis=1).mean(axis=1)
football_reduced_df['away_avg_opponent_rating_last_10'] = football_train_df.filter(regex='away_team_history_opponent_rating', axis=1).mean(axis=1)

print(football_reduced_df.shape)


(79055, 24)


In [120]:
print(football_reduced_df['home_avg_rating_last_10'].head())
print(football_reduced_df['away_avg_rating_last_10'].head())

id
11906497    5.496371
11983301    6.844712
11983471    7.449923
11883005    7.079634
11974168    8.236748
Name: home_avg_rating_last_10, dtype: float64
id
11906497    8.561214
11983301    7.620071
11983471    5.818128
11883005    5.983338
11974168    9.791657
Name: away_avg_rating_last_10, dtype: float64


## Model Selection

In [ ]:
# Split into test dataset and train dataset

### Test Linear Regression


In [ ]:
# linear regresion for blah blah

### Test K-Nearest Neighbors

In [ ]:
# Test KKN for classification

### Test XGBoost for Regression

In [ ]:
# code for a XGBoost regressor

### Models Comparison

In [ ]:
# compare metrics

## Model Training

In [ ]:
# prepare data for the model

# scaling data - additional encoder

# feature selection

# create model with hyperparameters

# train model

## Model Evaluation

In [ ]:
# test metrics classical

# show plots about results

In [ ]:
# scatter plot comparing real and predicted values

# line plot about real vs predicted

# box plot about real vs predicted

In [ ]:
# pickle the model
import pickle

# save the model to disk
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

***

# Business Questions

1. What is the risk level for unpayments based on age and region?


In [1]:
# generate a table or a plot or a business metric

2. What is average amount of unpayments? what is the segment of values where unpayments are appearin?

In [ ]:
# generate a table or a plot or a business metric